**The code in Practice is what I practiced.**

# Training a new tokenizer from an old one

## Practice

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

In [ ]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("code_search_net", "python")

In [5]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [6]:
print(raw_datasets["train"][321123]["whole_func_string"])

def get_limits(self):
        """
        Return all known limits for this service, as a dict of their names
        to :py:class:`~.AwsLimit` objects.

        Limits from:
        docs.aws.amazon.com/Route53/latest/DeveloperGuide/DNSLimitations.html

        :returns: dict of limit names to :py:class:`~.AwsLimit` objects
        :rtype: dict
        """
        if not self.limits:
            self.limits = {}
            for item in [self.MAX_RRSETS_BY_ZONE,
                         self.MAX_VPCS_ASSOCIATED_BY_ZONE]:
                self.limits[item["name"]] = AwsLimit(
                    item["name"],
                    self,
                    item["default_limit"],
                    self.warning_threshold,
                    self.critical_threshold,
                    limit_type='AWS::Route53::HostedZone',
                    limit_subtype=item["name"]
                )

        return self.limits


In [7]:
# Don't uncomment the following line unless your dataset is small!
# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

In [8]:
training_corpus = (
    raw_datasets["train"][i:i+1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

In [12]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


In [14]:
def get_training_corpus():
    return (
        raw_datasets["train"][i:i+1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

training_corpus = get_training_corpus()

In [15]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx:start_idx+1000]
        yield samples["whole_func_string"]

In [17]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [18]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [19]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [20]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [21]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [22]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'randn',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

In [25]:
tokenizer.save_pretrained("code-search-net-tokenizer")

('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
tokenizer.push_to_hub("code-search-net-tokenizer_NLP-course-chapter6-section1")

CommitInfo(commit_url='https://huggingface.co/BanUrsus/code-search-net-tokenizer_NLP-course-chapter6-section1/commit/73848d03dfef3ecb14adb02d445f42b3af8e4b41', commit_message='Upload tokenizer', commit_description='', oid='73848d03dfef3ecb14adb02d445f42b3af8e4b41', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
tokenizer = AutoTokenizer.from_pretrained("BanUrsus/code-search-net-tokenizer_NLP-course-chapter6-section1")

## Example

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

In [ ]:
raw_datasets["train"]

In [ ]:
print(raw_datasets["train"][123456]["whole_func_string"])

In [ ]:
# Don't uncomment the following line unless your dataset is small!
# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

In [ ]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

In [ ]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

In [ ]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

In [ ]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

In [ ]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

In [ ]:
# Replace "huggingface-course" below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

# Fast tokenizers' special powers (PyTorch)

## Practice

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Ban and I am in Manhattan."
encoding = tokenizer(example)
print(type(encoding))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [28]:
tokenizer.is_fast

True

In [29]:
encoding.is_fast

True

In [30]:
encoding.tokens()

['[CLS]',
 'My',
 'name',
 'is',
 'Ban',
 'and',
 'I',
 'am',
 'in',
 'Manhattan',
 '.',
 '[SEP]']

In [31]:
encoding.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None]

In [33]:
"""
    Try it out! 
    Create a tokenizer from the bert-base-cased and roberta-base checkpoints and tokenize ”81s” with them. 
    What do you observe? What are the word IDs?
"""
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
my_example = "81s"
bert_encoding = bert_tokenizer(my_example)
roberta_encoding = roberta_tokenizer(my_example)
print(bert_encoding.tokens())
print(bert_encoding.word_ids)
print("-----")
print(roberta_encoding.tokens())
print(roberta_encoding.word_ids)

['[CLS]', '81', '##s', '[SEP]']
<bound method BatchEncoding.word_ids of {'input_ids': [101, 5615, 1116, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}>
-----
['<s>', '81', 's', '</s>']
<bound method BatchEncoding.word_ids of {'input_ids': [0, 6668, 29, 2], 'attention_mask': [1, 1, 1, 1]}>


In [34]:
start, end = encoding.word_to_chars(3)
example[start:end]

'Ban'

In [139]:
"""
    Try it out! 
    Create your own example text and see if you can understand which tokens are associated with word ID, and also how to extract the character spans for a single word. 
    For bonus points, try using two sentences as input and see if the sentence IDs make sense to you.
"""
from transformers import AutoTokenizer

t_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
t_example = "It's not how much time you have, it's how you use it."
t_encoding = t_tokenizer(t_example)
print(t_encoding)
print(t_encoding.tokens())
print(t_encoding.word_ids())
t_start, t_end = t_encoding.word_to_chars(6)
print(t_example[t_start:t_end])

print("-----")

t_example = ["It's not how much time you have, it's how you use it.", "Don't judge a book by its cover."]
t_encoding = t_tokenizer(*t_example)
print(t_encoding)
print(t_encoding.tokens())
print(t_encoding.word_ids())

{'input_ids': [101, 1135, 112, 188, 1136, 1293, 1277, 1159, 1128, 1138, 117, 1122, 112, 188, 1293, 1128, 1329, 1122, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'It', "'", 's', 'not', 'how', 'much', 'time', 'you', 'have', ',', 'it', "'", 's', 'how', 'you', 'use', 'it', '.', '[SEP]']
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, None]
time
-----
{'input_ids': [101, 1135, 112, 188, 1136, 1293, 1277, 1159, 1128, 1138, 117, 1122, 112, 188, 1293, 1128, 1329, 1122, 119, 102, 1790, 112, 189, 3942, 170, 1520, 1118, 1157, 2267, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'It', "'", 's', 'not', 'how', 'much', 'time', 'you', 'have', ',', 'i

In [141]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My name is BanUrsus and I am in Manhattan.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.6863473,
  'index': 4,
  'word': 'Ban',
  'start': 11,
  'end': 14},
 {'entity': 'I-ORG',
  'score': 0.5376546,
  'index': 5,
  'word': '##U',
  'start': 14,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.43925196,
  'index': 6,
  'word': '##rs',
  'start': 15,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.41892707,
  'index': 7,
  'word': '##us',
  'start': 17,
  'end': 19},
 {'entity': 'I-LOC',
  'score': 0.9980684,
  'index': 12,
  'word': 'Manhattan',
  'start': 32,
  'end': 41}]

In [143]:
from transformers import pipeline
token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("My name is BanUrsus and I am in Manhattan.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.6863473,
  'word': 'Ban',
  'start': 11,
  'end': 14},
 {'entity_group': 'ORG',
  'score': 0.5376546,
  'word': '##U',
  'start': 14,
  'end': 15},
 {'entity_group': 'PER',
  'score': 0.42908952,
  'word': '##rsus',
  'start': 15,
  'end': 19},
 {'entity_group': 'LOC',
  'score': 0.9980684,
  'word': 'Manhattan',
  'start': 32,
  'end': 41}]

In [144]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is BanUrsus and I am in Manhattan."
inputs = tokenizer(example, return_tensors='pt')
outputs = model(**inputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [147]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 15])
torch.Size([1, 15, 9])


In [157]:
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

[0, 0, 0, 0, 4, 6, 4, 4, 0, 0, 0, 0, 8, 0, 0]


In [167]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [182]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != 'O':
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "index": idx, "word": tokens[idx]}
        )

print(results)

[{'entity': 'I-PER', 'score': 0.6863473057746887, 'index': 4, 'word': 'Ban'}, {'entity': 'I-ORG', 'score': 0.5376545786857605, 'index': 5, 'word': '##U'}, {'entity': 'I-PER', 'score': 0.43925198912620544, 'index': 6, 'word': '##rs'}, {'entity': 'I-PER', 'score': 0.41892707347869873, 'index': 7, 'word': '##us'}, {'entity': 'I-LOC', 'score': 0.9980685114860535, 'index': 12, 'word': 'Manhattan'}]


In [176]:
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 14),
 (14, 15),
 (15, 17),
 (17, 19),
 (20, 23),
 (24, 25),
 (26, 28),
 (29, 31),
 (32, 41),
 (41, 42),
 (0, 0)]

In [178]:
example[11:14]

'Ban'

In [187]:
results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "index": idx,
                "start": start,
                "end": end,
            }
        )

print(results)

[{'entity': 'I-PER', 'score': 0.6863473057746887, 'word': 'Ban', 'index': 4, 'start': 11, 'end': 14}, {'entity': 'I-ORG', 'score': 0.5376545786857605, 'word': '##U', 'index': 5, 'start': 14, 'end': 15}, {'entity': 'I-PER', 'score': 0.43925198912620544, 'word': '##rs', 'index': 6, 'start': 15, 'end': 17}, {'entity': 'I-PER', 'score': 0.41892707347869873, 'word': '##us', 'index': 7, 'start': 17, 'end': 19}, {'entity': 'I-LOC', 'score': 0.9980685114860535, 'word': 'Manhattan', 'index': 12, 'start': 32, 'end': 41}]


In [188]:
example[15:19]

'rsus'

In [208]:
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != 'O':
        label = label[2:]
        start, _ = offsets[idx]

        all_scores = []
        while (
            idx < len(predictions) 
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )

    idx += 1

print(results)

[{'entity': 'PER', 'score': 0.6863473057746887, 'word': 'Ban', 'start': 11, 'end': 14}, {'entity': 'PER', 'score': 0.4290895313024521, 'word': 'rsus', 'start': 15, 'end': 19}, {'entity': 'LOC', 'score': 0.9980685114860535, 'word': 'Manhattan', 'start': 32, 'end': 41}]


## Example

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
print(type(encoding))

In [ ]:
tokenizer.is_fast

In [ ]:
encoding.is_fast

In [ ]:
encoding.tokens()

In [ ]:
encoding.word_ids()

In [ ]:
start, end = encoding.word_to_chars(3)
example[start:end]

In [ ]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
from transformers import pipeline

token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

In [ ]:
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

In [ ]:
model.config.id2label

In [ ]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )

print(results)

In [ ]:
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

In [ ]:
example[12:14]

In [ ]:
results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

In [ ]:
example[33:45]

In [ ]:
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)